# Goals
The aim is to recreate FE versions of the test problems form the BICGStab paper and export them as mtx files.

## Set Up

In [59]:
using Gridap, LinearAlgebra

n = 8; Omega = (0.0,1.0,0.0,1.0);
Grid = CartesianDiscreteModel(Omega, (n,n))
# Grid has 4 corners and 4 edges
# "boundary" is all elements of the boundary of Omega
# Tags 1-4 are the corners and Tags 5-8 are the edges.
# Label each of the six sides differently
# boundary
Tags = get_face_labeling(Grid)
add_tag_from_tags!(Tags,"y=0",[1,2,5])
add_tag_from_tags!(Tags,"OtherEdges",[3,4,6,7,8])
# Piecewise linear Lagrange elements
order = 1; Elements = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(Grid,Elements;conformity=:H1,dirichlet_tags="y=0")
# Incorporating Homogeneous Dirichlet boundary conditions.
g(x) = 0.0
U = TrialFESpace(V,g)
# Triangulating and defining measures
degree = 2
Ω = Triangulation(Grid)
dΩ = Measure(Ω,degree)
# Setting up boundary triangulation/measure for Neumann conditions 
Γ = BoundaryTriangulation(Grid,tags="OtherEdges")
dΓ = Measure(Γ,degree)
# Setting up a simple test problem
f(x) = 1.0; h(x) = 3.0
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ + ∫( v*h )*dΓ
# Set up FE operator
op = AffineFEOperator(a,b,U,V)
# Extract Stiffness Matrix A
A=get_matrix(op)
eigen(Matrix(A)).values

72-element Vector{Float64}:
 0.03018854458905018
 0.13015869661768198
 0.2580940382369534
 0.32363449646715736
 0.40440555594065186
 0.6074817490683664
 0.6272406729045628
 0.6451495603184632
 0.7520522648022361
 0.8979375530735099
 0.9846668624131869
 1.0020966278020107
 1.0977663267055304
 ⋮
 3.3005520398401154
 3.310329341761117
 3.3819154022765274
 3.418448382928359
 3.452935112363834
 3.502261037792478
 3.522353392003811
 3.5258200837004834
 3.7051687302264877
 3.71025825360816
 3.8236088938001602
 3.8354756159661196